| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

This notebook is licensed under GPL 3.0. Please visit our [Github repo](https://github.com/edgi-govdata-archiving/ECHO-Cross-Program) for more information.

The notebook was collaboratively authored by EDGI following our [authorship protocol](https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/).

For more information about this project, visit https://www.environmentalenforcementwatch.org/

# Predicting Enforcement Actions by the US EPA

This notebook examines data from the EPA's Enforcement and Compliance History Online (ECHO) database (https://echo.epa.gov/). It includes information from EPA's programs covering air quality (the Clean Air Act, or CAA), water quality (the Clean Water Act, or CWA), drinking water (Safe Drinking Water Act, SDWA) and hazardous and other waste processing (the Resource Recovery and Conservation Act, or RCRA). 

ECHO data is available for facility violations as well as inspections and enforcement actions by EPA, state and other agencies. The data made accessible here runs from the present day (the database is refreshed weekly) back to 2001, which is when the EPA believes the data to be most reliable. It is available at the Congressional District level for a selected state, and for counties and zip codes of your choosing. 

The EPA program acronyms are:
- CAA = Clean Air Act
- CWA = Clean Water Act
- SDWIS = Safe Drinking Water Information System
- RCRA = Resource Conservation and Recovery Act
- TRI = Toxics Release Inventory
- GHG = Greenhouse Gas Reporting Program


## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# **Let's begin!**

Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

These first two cells give us access to some external Python code we will need.

### 1.  Bring in some code that is stored in a Github project, and run a few Python modules.

In [ ]:
!pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules
!pip install geopandas &>/dev/null;

print("Done!")

### 2. Let's get familiar with the database.

We will construct a query and look at some of the EPA ECHO data.

In [ ]:
from ECHO_modules.get_data import get_echo_data

# In the Python programming language, we use # to write comments in our code. 
# The computer doesn't process these, but they are important for communicating context!

# First, we'll write a SQL query. SQL = Structured Query Language. It is a standard format / syntax
# for asking databases for specific kinds of information.

# Our SQL query begins with a variable sql set to a string that is the query itself
sql = 'select * from "ECHO_EXPORTER" where "FAC_STATE" = \'IA\''
# This should be read as, select all columns from the ECHO_EXPORTER table (spreadsheet)
# where the FAC_STATE column is equal to IA (Iowa)

# This is the URL where we get the data from - a server hosted by Stony Brook University
#url= 'http://portal.gss.stonybrook.edu/echoepa/?query=' 

# The specific location of the data
#data_location=url+urllib.parse.quote_plus(sql) + '&pg'

print(sql) # Debugging

# Get the data. `get_data` is a behind the scenes function we wrote in another script and which
# was imported in Step 1 of this notebook. It helps us get data from the database!
facilities = get_echo_data(sql)

# Show the results
facilities

### 3. Wow, that's a lot of data! How can we work with it?
The next few cells introduce some common techniques for parsing and displaying data. We use the Pandas package of add-ons to Python.

In [ ]:
# Loop through and list all the columns in this table (spreadsheet)
for column in facilities.columns:
  display(column)

In [ ]:
# Show facilities "flagged" or regulated under the Clean Water Act (CWA)'s National
# Pollutant Discharge Elimination System (NPDES)
npdes = facilities.loc[facilities['NPDES_FLAG'] == 'Y']
npdes # Display the facilities
# In this case, we have created a new variable `npdes` to store just the facilities that are NPDES-regulated
# This allows us to work just with this sliver of the data in later cells.

In [ ]:
# Show the violations associated with these. How many times over the past 13 quarters
# have these facilities violated their permits?
npdes['CWA_13QTRS_EFFLNT_EXCEEDANCES']

In [ ]:
# As you can see, some are NaN (no value listed)
# So let's filter to just those with at least 1 effluent violation
violators = npdes.loc[npdes['CWA_13QTRS_EFFLNT_EXCEEDANCES'] > 0]
violators

In [ ]:
# These 800 or so facilities have exceeded their permits in the past 3 years.
# But has US EPA and its state counterparts taken any action (penalties, fines, warnings, etc)
# against them (in the past 5 years)? 

no_info = violators.loc[violators['CWA_FORMAL_ACTION_COUNT'].isna()] # Violators with no info about enforcement actions
violators_with_enforcement = violators.loc[violators['CWA_FORMAL_ACTION_COUNT'] > 0] # Violators with at least 1 enforcement action

print(len(no_info.index)) # Print the number of facilities with no info about enforcement actions
print ((len(violators_with_enforcement.index) / len(violators.index)) *100) # Print the share of violating facilities with enforcement actions

### 4. Construct your own query!
What sorts of variables (facility location, neighborhood demographics, etc.) predict whether or not enforcement actions have been taken?


In [ ]:
sql = '' # Your query here
# Some notes about syntax. Use "" double quotes around the names of tables and columns.
# Use single quotes '' around the entire query
# Use \' around strings that are a part of the query e.g. \'BATON ROUGE\'
results = get_echo_data(sql)

## More details on program data available in the ECHO database...

 **Inspections** or **evaluations** are usually the first phase in the enforcement pipeline, how many (but not all) violations are discovered.  **Violations** happen when a facilty is found to be out of compliance with an environmental law, often by violating the terms of its permit issued under that law.  **Enforcement actions** happen when the regulating agency decides to initiate a "case" against a firm for its violation of an environmental law, either by considering some administrative penalty such as a fine or by taking the violator to court. 

What's available for you to look at here:

https://github.com/edgi-govdata-archiving/ECHO-Oil-Gas/blob/main/slim_echoepa_a_schema.sql

**Hazardous and Other Waste**:
- RCRA_EVALUATIONS = Inspections under RCRA
- RCRA_VIOLATIONS = Violations of RCRA rules
- RCRA_ENFORCEMENTS = Enforcement actions taken by state agencies and the EPA

Here's where you can find definitions of terms used in the columns of RCRA data: 
https://echo.epa.gov/tools/data-downloads/rcrainfo-download-summary 

**Air**:
- ICIS-AIR_VIOLATION_HISTORY = CAA violations
- ICIS-AIR_FCES_PCES = Both state and federal CAA compliance evaluations
- ICIS-AIR_FORMAL_ACTIONS = CAA formal enforcement actions

Here's where you can find definitions of terms used in the columns of Clean Air Act data: 
https://echo.epa.gov/tools/data-downloads/icis-air-download-summary

**Air Emissions** (Combined air emissions data for stationary sources from four EPA air programs: National Emissions Inventory (NEI), Greenhouse Gas Reporting Program (GHGRP), Toxic Release Inventory (TRI), and Clean Air Markets (CAMD)):
- Greenhouse Gases via POLL_RPT_COMBINED_EMISSIONS
- Toxic Releases via POLL_RPT_COMBINED_EMISSIONS

Here's where you can find definitions of terms used in the columns of this data: https://echo.epa.gov/tools/data-downloads/air-emissions-download-summary

**Water** (National Pollutant Discharge Elimination System, or NPDES):
- NPDES_QNCR_HISTORY = CWA Quarterly Non-Compliance History
- NPDES_INSPECTIONS = CWA Inspections
- NPDES_FORMAL_ENFORCEMENT_ACTIONS = CWA Enforcements

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/icis-npdes-download-summary

**Drinking Water** (Safe Drinking Water Act, or SDWA):
- SDWA_PUB_WATER_SYSTEMS = Public Water Systems under Safe Drinking Water Act
- SDWA_SITE_VISITS = Safe Drinking Water Site Visits
- SDWA_VIOLATIONS = Safe Drinking Water Violations
- SDWA_SERIOUS_VIOLATORS = Safe Drinking Water Serious Violators
- SDWA_ENFORCEMENTS = Safe Drinking Water Enforcements
- SDWA_RETURN_TO_COMPLIANCE = Safe Drinking Water Return to Compliance

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/sdwa-download-summary